# Correspondence-driven plane-based M3C2 (PBM3C2) with pre-segmented planes
Implemented in: py4dgeo.pbm3c2


**Related publication**
Zahs, V., Winiwarter, L., Anders, K., Williams, J.G., Rutzinger, M. & Höfle, B. (2022): Correspondence-driven plane-based M3C2 for lower uncertainty in 3D topographic change quantification. ISPRS Journal of Photogrammetry and Remote Sensing, 183, pp. 541-559. DOI: [10.1016/j.isprsjprs.2021.11.018](https://doi.org/10.1016/j.isprsjprs.2021.11.018).

## **Method description**
In this notebook, we present how the *Correspondence-driven plane-based M3C2* (PB-M3C2, [Zahs et al., 2022] algorithm for point cloud distance computation using the `py4dgeo` package.

The concept and method of PBM3C2 are explained in this scientific talk:

<a href="https://youtu.be/5pjkpajsRNU" target="_blank"><img src="https://github.com/3dgeo-heidelberg/py4dgeo/blob/main/doc/img/thumb_youtube_zahs_isprs2022.png?raw=true" alt="" width="400" /></a>

In the current implementation of PBM3C2, a plane segmentation outside py4dgeo (e.g., using CloudCompare or other tools) is required. As PB-M3C2 is a learning algorithm, it requires user-labelled input data in the process, which can be created in graphical software, such as CloudCompare.

In [1]:
import py4dgeo
import numpy as np
import pooch

In this notebook, we use a dataset of synthetic planes, which is downloaded from the py4dgeo data repository:

In [2]:
p = pooch.Pooch(base_url="doi:10.5281/zenodo.16751963/", path=pooch.os_cache("py4dgeo"))
p.load_registry_from_doi()

try:
    # Download and extract the dataset
    p.fetch("pbm3c2.zip", processor=pooch.Unzip(members=["pbm3c2"]))

    # Define path to the extracted data
    data_path = p.path / "pbm3c2.zip.unzip" / "pbm3c2"
    print(f"Data path: {data_path}")

    # Read XYZ files from the extracted directory
    epoch0_path = str(data_path / "epoch0.xyz")
    epoch1_path = str(data_path / "epoch1.xyz")
    training_segments_path = str(data_path / "epoch_extended_y.csv")

except Exception as e:
    print(f"Failed to download or extract data: {e}")

ValueError: Archive with doi:10.5281/zenodo.16751963 not found (see https://doi.org/10.5281/zenodo.16751963). Is the DOI correct?

We are reading the two input epochs from XYZ files which contain a total of four columns: X, Y and Z coordinates, as well a segment ID mapping each point to a plane and normal vector components in X, Y and Z. The `read_from_xyz` functionality allows us to read additional data columns through its `additional_dimensions` parameter. It is expecting a dictionary that maps the column index to a column name.

In [ ]:
epoch0 = py4dgeo.epoch.read_from_xyz(
    epoch0_path,
    additional_dimensions={3: "segment_id", 4: "N_x", 5: "N_y", 6: "N_z"},
    delimiter=" ",
)
epoch1 = py4dgeo.epoch.read_from_xyz(
    epoch1_path,
    additional_dimensions={3: "segment_id", 4: "N_x", 5: "N_y", 6: "N_z"},
    delimiter=" ",
)

The point cloud data we use here consists of 100 planar segments, with 70 used for training and 30 for application.

In [ ]:
n_planes = 100
n_train = int(0.7 * n_planes)
train_ids = np.arange(n_train)
apply_ids = np.arange(n_train, n_planes)

We instantiate an instance of the algorithm class. Here, you can set the registration error for the input point clouds.

In [ ]:
alg = py4dgeo.PBM3C2(reg_error=0.01)

The algorithm requires the user to provide a labeled training dataset **correspondences_file** to learn how to match the segments. This csv file contains three columns: the first two are the plane segment_id from epoch 1 and epoch 2, and the third is a label (1 for a correct match, 0 for an incorrect one).

In [ ]:
correspondences_df = alg.compute(
    epoch0=epoch0,
    epoch1=epoch1,
    correspondences_file=training_segments_path,
    apply_ids=apply_ids,
    search_radius=5.0,
)

In [ ]:
print(correspondences_df.head())

In [ ]:
distances = correspondences_df["distance"]
uncertainties = correspondences_df["uncertainty"]

## References
* Zahs, V., Winiwarter, L., Anders, K., Williams, J.G., Rutzinger, M. & Höfle, B. (2022): Correspondence-driven plane-based M3C2 for lower uncertainty in 3D topographic change quantification. ISPRS Journal of Photogrammetry and Remote Sensing, 183, pp. 541-559. DOI: [10.1016/j.isprsjprs.2021.11.018](https://doi.org/10.1016/j.isprsjprs.2021.11.018).